In [16]:
from concurrent.futures import ProcessPoolExecutor, wait, FIRST_COMPLETED
import sklearn.model_selection
import numpy as np
import pandas as pd
import yaml

In [17]:
from irec.environment.loader import FullData

In [18]:
from irec.recommendation.agents import SimpleAgent
from irec.recommendation.agents.action_selection_policies import ASPEGreedy, ASPGreedy

In [19]:
from irec.recommendation.hyperoptimization.grid_search import GridSearch

In [20]:
from irec.offline_experiments.evaluation_policies import FixedInteraction
from irec.offline_experiments.metric_evaluators import UserCumulativeInteraction
from irec.offline_experiments.metrics import Hits

In [21]:
from irec.recommendation.agents.value_functions import *
import sys
sys.path.append("..")
from wscb.value_function import WSCB

In [22]:
# Dataset
dataset = {
    'path': "../dataset/MovieLens 100k/ratings.csv",
    'random_seed': 0,
    'file_delimiter': ",",
    'skip_head': True
}
# Splitting
splitting = {'strategy': "temporal", 'train_size': 0.8, 'test_consumes': 5}
validation = {'validation_size': 0.2}
# Loader
loader = FullData(dataset, splitting, validation)
train_dataset, test_dataset, x_validation, y_validation = loader.process()


Applying splitting strategy: temporal

Test shape: (16892, 4)
Train shape: (83108, 4)

Generating x_validation and y_validation: 
Test shape: (16522, 4)
Train shape: (66586, 4)


In [23]:
def x(start, end, step):
    return list(np.around(np.linspace(start, end, step), 3))

agents_search_parameters = {
    "UCB": {"c": x(0.001, 1, 10)},
    "ThompsonSampling": {"alpha_0": x(0.1, 1, 5), "beta_0": x(1, 100, 5)},
    "EGreedy": {"epsilon": x(0.001, 1, 10)},
    "LinearUCB": {"alpha": x(0.1, 1, 5), "item_var": [0.01], "iterations": [20], "num_lat": [10, 15, 20], "stop_criteria": [0.0009], "user_var": [0.01], "var": [0.05]},
    # "GLM_UCB": {"c": [0.1, 0.5, 1, 2, 4, 8], "item_var": [0.01], "iterations": [20], "num_lat": [5, 10, 20], "stop_criteria": [0.0009], "user_var": [0.01], "var": [0.05]},
    # "NICF": {"batch": [256], "clip_param": [0.2], "dropout_rate": [0.01, 0.1], "gamma": [0.0], "inner_epoch": [200], "latent_factor": [10], "learning_rate": [0.001], "num_blocks": [1, 2, 3], "num_heads": [1, 2, 3], "restore_model": [False], "rnn_layer": [1, 2], "time_step": [100], "training_epoch": [10000]},
    # "PTS": {"num_lat": [5, 10], "num_particles": [5], "var": x(0.01, 0.5, 4), "var_u": x(0.01, 0.5, 4), "var_v": x(0.3, 5.0, 3)},
    # "ICTRTS": {"num_lat": [5, 10, 20], "num_particles": [2, 5, 10]},
    # "ClusterBandit": {"B": [5], "C": [0.5], "D": [3], "num_clusters": [4, 8], "num_lat": [5, 10, 20]},
    "WSCB": {"alpha": x(0.1, 1, 10), "num_lat": [5, 10, 20]}
}

In [24]:
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=False)

In [ ]:
tasks = 8
results = []

with ProcessPoolExecutor(max_workers=tasks) as executor:
    futures = set()

    for agent_name, parameters_range in agents_search_parameters.items():
        parameters_range = list(sklearn.model_selection.ParameterGrid(parameters_range))

        for agent_parameters in parameters_range:
            
            value_function = eval(agent_name)(**agent_parameters)
            action_selection_policy = ASPEGreedy(**agent_parameters) if agent_name == "EGreedy" else ASPGreedy()
            agent = SimpleAgent(value_function, action_selection_policy, name=agent_name)
            
            f = executor.submit(eval_policy.evaluate, agent, x_validation, y_validation)
            futures.add(f)
            results.append({
                "name": agent_name,
                "prmt": agent_parameters,
                "result": f
            })
        
            if len(futures) >= tasks:
                completed, futures = wait(futures, return_when=FIRST_COMPLETED)

for i, r in enumerate(results):
    results[i]["result"] = r["result"].result()

In [26]:
# Cumulative Evaluation Setup
evaluator = UserCumulativeInteraction(
    ground_truth_dataset=y_validation,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[100],
    relevance_evaluator_threshold=3.99
)

In [27]:
cumulative_results = []
for result in results:
    agent_name, agent_parameters, agent_results = result.values()
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results[0])
    cumulative_results.append({
        "Agent Name": agent_name,
        "Parameters": str(agent_parameters),
        "Top-Hits": np.mean(list(hits_values[0].values()))
    })

Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.36 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.32 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.36 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.30 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.33 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.33 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.32 seconds executing Hits metric
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.37 seconds executing Hits metric
Computin

In [28]:
df_cumulative = pd.DataFrame(cumulative_results)
df_cumulative

,Agent Name,Parameters,Top-Hits
0,UCB,{'c': 0.001},15.092715
1,UCB,{'c': 0.112},14.715232
2,UCB,{'c': 0.223},14.668874
3,UCB,{'c': 0.334},14.324503
4,UCB,{'c': 0.445},13.960265
...,...,...,...
100,WSCB,"{'alpha': 0.9, 'num_lat': 10}",26.218543
101,WSCB,"{'alpha': 0.9, 'num_lat': 20}",25.874172
102,WSCB,"{'alpha': 1.0, 'num_lat': 5}",24.682119
103,WSCB,"{'alpha': 1.0, 'num_lat': 10}",26.185430


In [29]:
df_cumulative.groupby("Agent Name").apply(lambda gp: display(gp.sort_values("Top-Hits", ascending=False)))

,Agent Name,Parameters,Top-Hits
35,EGreedy,{'epsilon': 0.001},14.993377
36,EGreedy,{'epsilon': 0.112},14.410596
37,EGreedy,{'epsilon': 0.223},13.735099
38,EGreedy,{'epsilon': 0.334},12.589404
39,EGreedy,{'epsilon': 0.445},11.112583
40,EGreedy,{'epsilon': 0.556},10.152318
41,EGreedy,{'epsilon': 0.667},9.059603
42,EGreedy,{'epsilon': 0.778},7.430464
43,EGreedy,{'epsilon': 0.889},5.271523
44,EGreedy,{'epsilon': 1.0},3.384106


,Agent Name,Parameters,Top-Hits
53,LinearUCB,"{'alpha': 0.3, 'item_var': 0.01, 'iterations':...",23.370861
50,LinearUCB,"{'alpha': 0.2, 'item_var': 0.01, 'iterations':...",23.357616
56,LinearUCB,"{'alpha': 0.4, 'item_var': 0.01, 'iterations':...",23.350993
47,LinearUCB,"{'alpha': 0.1, 'item_var': 0.01, 'iterations':...",23.337748
71,LinearUCB,"{'alpha': 0.9, 'item_var': 0.01, 'iterations':...",23.331126
59,LinearUCB,"{'alpha': 0.5, 'item_var': 0.01, 'iterations':...",23.324503
62,LinearUCB,"{'alpha': 0.6, 'item_var': 0.01, 'iterations':...",23.304636
74,LinearUCB,"{'alpha': 1.0, 'item_var': 0.01, 'iterations':...",23.298013
68,LinearUCB,"{'alpha': 0.8, 'item_var': 0.01, 'iterations':...",23.291391
65,LinearUCB,"{'alpha': 0.7, 'item_var': 0.01, 'iterations':...",23.284768


,Agent Name,Parameters,Top-Hits
29,ThompsonSampling,"{'alpha_0': 0.775, 'beta_0': 100.0}",18.715232
34,ThompsonSampling,"{'alpha_0': 1.0, 'beta_0': 100.0}",18.615894
14,ThompsonSampling,"{'alpha_0': 0.1, 'beta_0': 100.0}",18.576159
24,ThompsonSampling,"{'alpha_0': 0.55, 'beta_0': 100.0}",18.430464
19,ThompsonSampling,"{'alpha_0': 0.325, 'beta_0': 100.0}",18.390728
13,ThompsonSampling,"{'alpha_0': 0.1, 'beta_0': 75.25}",18.364238
18,ThompsonSampling,"{'alpha_0': 0.325, 'beta_0': 75.25}",18.364238
23,ThompsonSampling,"{'alpha_0': 0.55, 'beta_0': 75.25}",18.178808
33,ThompsonSampling,"{'alpha_0': 1.0, 'beta_0': 75.25}",18.158940
28,ThompsonSampling,"{'alpha_0': 0.775, 'beta_0': 75.25}",18.112583


,Agent Name,Parameters,Top-Hits
0,UCB,{'c': 0.001},15.092715
1,UCB,{'c': 0.112},14.715232
2,UCB,{'c': 0.223},14.668874
3,UCB,{'c': 0.334},14.324503
4,UCB,{'c': 0.445},13.960265
5,UCB,{'c': 0.556},13.364238
6,UCB,{'c': 0.667},13.271523
7,UCB,{'c': 0.778},12.900662
8,UCB,{'c': 0.889},12.443709
9,UCB,{'c': 1.0},12.324503


,Agent Name,Parameters,Top-Hits
91,WSCB,"{'alpha': 0.6, 'num_lat': 10}",26.311258
85,WSCB,"{'alpha': 0.4, 'num_lat': 10}",26.238411
97,WSCB,"{'alpha': 0.8, 'num_lat': 10}",26.225166
100,WSCB,"{'alpha': 0.9, 'num_lat': 10}",26.218543
88,WSCB,"{'alpha': 0.5, 'num_lat': 10}",26.218543
103,WSCB,"{'alpha': 1.0, 'num_lat': 10}",26.185430
94,WSCB,"{'alpha': 0.7, 'num_lat': 10}",26.158940
82,WSCB,"{'alpha': 0.3, 'num_lat': 10}",26.132450
98,WSCB,"{'alpha': 0.8, 'num_lat': 20}",25.980132
79,WSCB,"{'alpha': 0.2, 'num_lat': 10}",25.874172


""


In [30]:
best_params = df_cumulative.groupby("Agent Name").apply(lambda gp: gp.sort_values("Top-Hits", ascending=False).iloc[0])
best_params

,Agent Name,Parameters,Top-Hits
Agent Name,,,
EGreedy,EGreedy,{'epsilon': 0.001},14.993377
LinearUCB,LinearUCB,"{'alpha': 0.3, 'item_var': 0.01, 'iterations':...",23.370861
ThompsonSampling,ThompsonSampling,"{'alpha_0': 0.775, 'beta_0': 100.0}",18.715232
UCB,UCB,{'c': 0.001},15.092715
WSCB,WSCB,"{'alpha': 0.6, 'num_lat': 10}",26.311258
